In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display
from scipy.stats import mannwhitneyu, wilcoxon

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
from lib.sych.plot_helper import imshow_dataset_by_mouse, imshow_dataset_by_session
from lib.sych.metric_helper import metric_by_session, metric_by_selector

%load_ext autoreload
%autoreload 2

Appended root directory /media/aleksejs/DataHDD/work/codes/comp-neuro/analysis-mesoscopic/pub-2020-exploratory-analysis


In [2]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
params['root_path_data'] = './'

In [3]:
dataDB = DataFCDatabase(params)

Searching for data files


In [4]:
ds = DataStorage('sych_result_bulk_metrics.h5')

In [5]:
mc = MetricCalculator(serial=True, verbose=False)

# 1. Bulk Mean vs Time

* Avg over mice
* All-region-all-types
* Expert vs Naive
* Go/NOGO/MISS/FA

In [8]:
none2all = lambda x: x if x is not None else 'All'

def metric_mouse_bulk_vs_time(dataDB, mc, ds, metricName, metricSettings=None, sweepSettings=None):
    for mousename in dataDB.mice:
        for datatype in dataDB.get_data_types(mousename):
            for performance in [None, 'naive', 'expert']:
                for trialType in [None] + dataDB.get_trial_type_names(mousename):
                    dataName = '_'.join([metricName, 'time', datatype, none2all(performance), none2all(trialType)])
                    zscoreDim = 'rs' if datatype == 'raw' else None
                    
                    print(mousename, dataName)
                    
                    metric_by_selector(dataDB, mc, ds, {'mousename' : mousename}, metricName, 's',
                                       dataName=dataName, datatype=datatype, trialType=trialType,
                                       performance=performance,
                                       zscoreDim=zscoreDim,
                                       metricSettings=metricSettings,
                                       sweepSettings=sweepSettings)
                    
                    
def plot_metric_bulk_vs_time(dataDB, mc, ds, metricName):
    t = np.arange(0, 8, 1/20)
    dummyMouseName = 'mvg_4'
    
    dfAll = ds.list_dsets_pd()
    for datatype in dataDB.get_data_types(dummyMouseName):    
        for performance in [None, 'naive', 'expert']:
            for trialType in [None] + dataDB.get_trial_type_names(dummyMouseName):
                dataName = '_'.join([metricName, 'time', datatype, none2all(performance), none2all(trialType)])
                dfThis = dfAll[dfAll['name'] == dataName]
                
                print(dataName)
                
                plt.figure()
                for idx, row in dfThis.iterrows():
                    dataThis = ds.get_data(row['dset'])
                    
                    if datatype == 'raw':
                        nTrialThis = dataDB.get_ntrial_bytype({'mousename' : row['mousename']}, trialType=trialType, performance=performance)
                        dataThis *= np.sqrt(48*nTrialThis)
                        print('--', row['mousename'], nTrialThis)
                        
                    
                    plt.plot(t, dataThis, label=row['mousename'])
                    
                plt.legend()
                plt.savefig('pics/'+ dataName + '.pdf')
                plt.close()      

In [7]:
metric_mouse_bulk_vs_time(dataDB, mc, ds, 'mean')

mvg_9 mean_time_bn_session_All_All
mvg_9 mean_time_bn_session_All_iGO
mvg_9 mean_time_bn_session_All_iNOGO
mvg_9 mean_time_bn_session_All_iFA
mvg_9 mean_time_bn_session_All_iMISS
mvg_9 mean_time_bn_session_naive_All
mvg_9 mean_time_bn_session_naive_iGO
mvg_9 mean_time_bn_session_naive_iNOGO
mvg_9 mean_time_bn_session_naive_iFA
mvg_9 mean_time_bn_session_naive_iMISS
mvg_9 mean_time_bn_session_expert_All
mvg_9 mean_time_bn_session_expert_iGO
mvg_9 mean_time_bn_session_expert_iNOGO
mvg_9 mean_time_bn_session_expert_iFA
mvg_9 mean_time_bn_session_expert_iMISS
mvg_9 mean_time_bn_trial_All_All
mvg_9 mean_time_bn_trial_All_iGO
mvg_9 mean_time_bn_trial_All_iNOGO
mvg_9 mean_time_bn_trial_All_iFA
mvg_9 mean_time_bn_trial_All_iMISS
mvg_9 mean_time_bn_trial_naive_All
mvg_9 mean_time_bn_trial_naive_iGO
mvg_9 mean_time_bn_trial_naive_iNOGO
mvg_9 mean_time_bn_trial_naive_iFA
mvg_9 mean_time_bn_trial_naive_iMISS
mvg_9 mean_time_bn_trial_expert_All
mvg_9 mean_time_bn_trial_expert_iGO
mvg_9 mean_time_bn

In [10]:
plot_metric_bulk_vs_time(dataDB, mc, ds, 'mean')

mean_time_bn_session_All_All
mean_time_bn_session_All_iGO
mean_time_bn_session_All_iNOGO
mean_time_bn_session_All_iFA
mean_time_bn_session_All_iMISS
mean_time_bn_session_naive_All
mean_time_bn_session_naive_iGO
mean_time_bn_session_naive_iNOGO
mean_time_bn_session_naive_iFA
mean_time_bn_session_naive_iMISS
mean_time_bn_session_expert_All
mean_time_bn_session_expert_iGO
mean_time_bn_session_expert_iNOGO
mean_time_bn_session_expert_iFA
mean_time_bn_session_expert_iMISS
mean_time_bn_trial_All_All
mean_time_bn_trial_All_iGO
mean_time_bn_trial_All_iNOGO
mean_time_bn_trial_All_iFA
mean_time_bn_trial_All_iMISS
mean_time_bn_trial_naive_All
mean_time_bn_trial_naive_iGO
mean_time_bn_trial_naive_iNOGO
mean_time_bn_trial_naive_iFA
mean_time_bn_trial_naive_iMISS
mean_time_bn_trial_expert_All
mean_time_bn_trial_expert_iGO
mean_time_bn_trial_expert_iNOGO
mean_time_bn_trial_expert_iFA
mean_time_bn_trial_expert_iMISS
mean_time_raw_All_All
-- mvg_8 5205
-- mvg_4 6540
-- mvg_9 12190
-- mvg_7 8637
mean_ti